<a href="https://colab.research.google.com/github/rihanuch/rihanuch-IIC3633-2020/blob/master/practicos/04_SlopeOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.youtube.com" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# Práctico Sistemas Recomendadores: pyreclab - Slope One

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Rodrigo Hanuch

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**
- Es de filtrado colaborativo debido a que se basa en ratings entregados para un item-base CF y propone mejoras en rendimiento para este
- Es basado en items debido a que usa una diferencia entre los promedios de los elementos
- Como columnas toma a los ratings de los clientes y hace una regresión con un solo parametro (f(x)=x+b)
- Si se agrega un nuevo rating se debe recalcular la media ponderada que se le entrego



#Configuración Inicial

**Paso 1:** Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [14]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    800      0 --:--:-- --:--:-- --:--:--   800
100 1546k  100 1546k    0     0  2032k      0 --:--:-- --:--:-- --:--:-- 2032k


In [15]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1102      0 --:--:-- --:--:-- --:--:--  1102
100  385k  100  385k    0     0   656k      0 --:--:-- --:--:-- --:--:-- 12.5M


In [16]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1141      0 --:--:-- --:--:-- --:--:--  1137
100  230k  100  230k    0     0   321k      0 --:--:-- --:--:-- --:--:--  321k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

**Paso 2:** Instalamos pyreclab y seaborn utilizando pip.

In [17]:
!pip install pyreclab --upgrade

Requirement already up-to-date: pyreclab in /usr/local/lib/python3.6/dist-packages (0.1.14)


**Paso 3:** Hacemos los imports necesarios para este práctico.

In [18]:
import pyreclab
import numpy as np
import pandas as pd

# El dataset

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

**Paso 4:** Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [19]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# Slope One

**Paso 5:** Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [20]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [21]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:**
Realmente no entrena el modelo debido a no "aprende" sino que ordena la información y calculas las medias ponderadas


**Paso 6:** Llego la hora de predecir el rating.

In [27]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [26]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

**Paso 7:** Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [28]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [29]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
info_file.loc[recmovies_slopeone]['title']

movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 3 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:**

Se puede ver una clara diferncia, ya que no se encuentran recomendaciones en común

In [38]:
# Escribe el nuevo codigo aqui
new_recommendation = mySlopeOne.recommend("20", 10)
print('Lista de items según ID:', new_recommendation)

new_recmovies_slopeone = np.array(new_recommendation).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
info_file.loc[new_recmovies_slopeone]['title']

Lista de items según ID: ['1599', '1536', '509', '508', '502', '500', '499', '498', '497', '492']


movieid
1599         Someone Else's America (1995)
1536                  Aiqing wansui (1994)
509                    My Left Foot (1989)
508     People vs. Larry Flynt, The (1996)
502                         Bananas (1971)
500                   Fly Away Home (1996)
499           Cat on a Hot Tin Roof (1958)
498              African Queen, The (1951)
497                Bringing Up Baby (1938)
492                    East of Eden (1955)
Name: title, dtype: object

## Actividad Opcional 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:**

In [ ]:
# Escribe el nuevo codigo aqui